In [1]:
from serde import serde
from serde.json import to_json, from_json
from dataclasses import dataclass
from time import sleep
from scrapping import Voiture
import sklearn
import numpy as np
import pandas as pd
import conversion_df

# Récuperation du json et conversion en quelque chose d'utilisable

# Conversion en base de données panda et nettoyage

## Remarques sur les variables :

- modele : enlever les espaces, ou supprimer les variables peu occurentes (<=1)
- carburant : même remarque
- même remarque pour toutes les variables textuelles
- utilisation précédente : regrouper par pro/loueur/part/ ne sait pas ?

## Création de dummys


In [2]:
df = conversion_df.data_frame_pandas('donnees_citroen.json')

In [3]:
df.dtypes


marque                    object
modele                    object
carburant                 object
prix                       int64
kilometrage              float64
garantie_kilometrage      object
boite_de_vitesse          object
transmission             float64
couleur                   object
garantie                  object
date_mise_circulation      int64
puissance                float64
silhouette                object
nb_places                float64
utilisation_prec          object
puissance_fiscale        float64
critair                  float64
ptac                     float64
nb_portes                float64
dtype: object

In [4]:
df.ptac

0      1835.0
1      2300.0
2      1735.0
3      1540.0
4      1600.0
        ...  
595    1795.0
596    1610.0
597    1610.0
598    1610.0
599    1610.0
Name: ptac, Length: 600, dtype: float64

In [5]:
df.convert_dtypes().dtypes

marque                   string
modele                   string
carburant                string
prix                      Int64
kilometrage               Int64
garantie_kilometrage     string
boite_de_vitesse         string
transmission              Int64
couleur                  string
garantie                 string
date_mise_circulation     Int64
puissance                 Int64
silhouette               string
nb_places                 Int64
utilisation_prec         string
puissance_fiscale         Int64
critair                   Int64
ptac                      Int64
nb_portes                 Int64
dtype: object

## Passage sous numpy + ML

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
import conversion_df
import pandas as pd

conversion_df = reload(conversion_df)

In [7]:
trp = conversion_df.data_frame_pandas('donnees_temp.json')
df = conversion_df.data_frame_pandas('donnees_peugeot.json')

In [8]:
trp

,marque,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes
0,Peugeot,2008,Essence,30000,60000,garantie,Automatique,2,Noir,SPOTICAR PREMIUM,2020,130,SUV-4x4,5,Ex-Particulier,6,2,1500,3


In [8]:
trp_fin = pd.concat([df,trp], ignore_index=True)
trp_fin_cat = conversion_df.data_frame_dummies(trp_fin)

In [9]:
trp_fin_cat

,prix,kilometrage,date_mise_circulation,puissance,nb_places,puissance_fiscale,critair,ptac,nb_portes,Peugeot,...,AUTOEXPERT,SPOTICAR ADVANCED,SPOTICAR ESSENTIAL,SPOTICAR PREMIUM,Ex-Auto-ï¿½cole,Ex-Import,Ex-Loueur,Ex-Particulier,Ex-Sociï¿½tï¿½,Vï¿½hicule de direction
0,20290,75967,2019,130.0,5.0,6.0,2.0,1890.0,5.0,1,...,0,0,0,1,0,0,1,0,0,0
1,31900,29066,2021,130.0,5.0,7.0,2.0,1875.0,5.0,1,...,0,0,0,1,0,0,1,0,0,0
2,33920,31997,2021,130.0,5.0,7.0,1.0,1875.0,5.0,1,...,0,0,0,1,0,0,1,0,0,0
3,21980,40952,2021,100.0,5.0,5.0,1.0,1875.0,5.0,1,...,0,0,0,1,0,0,1,0,0,0
4,22990,7901,2021,100.0,5.0,5.0,1.0,1875.0,5.0,1,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,31490,2535,2022,131.0,5.0,7.0,2.0,1770.0,5.0,1,...,0,0,0,1,0,1,0,0,0,0
597,37490,21164,2022,131.0,5.0,7.0,2.0,2000.0,5.0,1,...,0,0,0,1,0,0,1,0,0,0
598,36490,10,2022,145.0,3.0,7.0,2.0,1875.0,5.0,1,...,0,0,0,1,0,0,1,0,0,0
599,38990,10,2022,177.0,6.0,7.0,2.0,2790.0,5.0,1,...,0,0,0,1,0,0,1,0,0,0


In [27]:
X_pred = trp_fin_cat.loc[:, trp_fin_cat.columns != 'prix'].to_numpy()
y_pred = trp_fin_cat['prix'].to_numpy()

In [28]:
df_cat = conversion_df.data_frame_dummies(df)
X = df_cat.loc[:, df_cat.columns != 'prix'].to_numpy()
y = df_cat['prix'].to_numpy()

In [29]:
trp_fin_cat.columns.values

array(['prix', 'kilometrage', 'date_mise_circulation', 'puissance',
       'nb_places', 'puissance_fiscale', 'critair', 'ptac', 'nb_portes',
       'Peugeot', '108', '2008', '208', '3008', '308', '308 SW', '4008',
       '5008', '508', '508 SW', 'BOXER', 'EXPERT', 'PARTNER', 'RIFTER',
       'TRAVELLER', 'garanti', 'non garanti', 'Diesel', 'Electrique',
       'Essence', 'Hybride rechargeable', 'Automatique', 'Manuelle',
       'Blanc', 'Bleu', 'Brun', 'Gris', 'Jaune', 'Noir', 'Orange',
       'Rouge', 'Sable', 'Vert', 'Berline', 'Break', 'Citadine',
       'Familiale', 'SUV-4x4', 'Utilitaire', 2.0, 4.0, 'AUTOEXPERT',
       'SPOTICAR ADVANCED', 'SPOTICAR ESSENTIAL', 'SPOTICAR PREMIUM',
       'Ex-Auto-ï¿½cole', 'Ex-Import', 'Ex-Loueur', 'Ex-Particulier',
       'Ex-Sociï¿½tï¿½', 'Vï¿½hicule de direction'], dtype=object)

In [30]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y)

In [31]:
en = ElasticNet()
en_gs = GridSearchCV(
    en,
    {
        "alpha": [2 ** p  for p in range(-6, 6)],
        "l1_ratio": (0.01, 0.25, 0.5, 0.75, 1),
    }
)
en_gs.fit(X_tr, y_tr) #problème vient du fit
en_df = pd.DataFrame(en_gs.cv_results_)
        

c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.484e+08, tolerance: 2.731e+06
  model = cd_fast.enet_coordinate_descent(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.787e+08, tolerance: 2.769e+06
  model = cd_fast.enet_coordinate_descent(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

In [43]:
en_gs.best_params_, en_gs.best_score_

({'alpha': 16, 'l1_ratio': 1}, 0.9104061866860143)

In [41]:
pd.DataFrame(en_gs.best_estimator_.predict(X_pred)).iloc[-1][0].round()

23278.0

In [48]:
y_pred

array([20290, 31900, 33920, 21980, 22990, 21990, 34390, 17890, 32990,
       20490, 18990, 11990, 26290, 33490, 17490, 32490, 25990, 26590,
       26290, 22490, 17690, 34490, 36990, 38790, 18990, 32590, 28990,
       25290, 16990, 22190, 23980, 33790, 38490, 22390, 22490, 26990,
       29990, 38980, 25990, 31690, 24590, 35990, 38290, 28790, 32490,
       26190, 34490, 23790, 26299, 38290, 35990, 21490, 25990, 16990,
       31290, 35690, 22290, 46790, 15490, 28980, 21990, 21990, 33091,
       30790, 40490, 28490, 36390, 17990, 39790, 17990, 38490, 23220,
       22190, 59980, 38990, 24790, 23290, 32480, 20480, 34990, 24590,
       27790, 38980, 28490, 23990, 26590, 22190, 15480, 23390, 34490,
       52890, 29290, 27590, 27990, 26490, 34490, 32490, 15290, 35690,
       27690, 35690, 39990, 42580, 36490, 26190, 32390, 35990, 28690,
       32300, 28990, 24990, 26590, 19290, 29990, 42590, 46290, 31580,
       52190, 42590, 37990, 28590, 19290, 39390, 39990, 39690, 26990,
       40490, 38280,

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

KNN

In [ ]:
knr = KNeighborsRegressor()
knr_gs = GridSearchCV(
    knr,
    {
        "n_neighbors": range(5, 15),
        "weights": ("uniform", "distance"),
    }
)

In [ ]:
knr_gs.fit(X_tr, y_tr)
knr_df = pd.DataFrame(knr_gs.cv_results_)
knr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000796,0.000398,0.036902,0.053869,5,uniform,"{'n_neighbors': 5, 'weights': 'uniform'}",0.237296,0.138595,0.085176,0.199244,0.242035,0.180469,0.060343,11
1,0.000596,0.000486,0.010615,0.001336,5,distance,"{'n_neighbors': 5, 'weights': 'distance'}",0.310453,0.242396,0.161994,0.256855,0.352684,0.264876,0.064718,10
2,0.000599,0.000489,0.006581,0.000797,6,uniform,"{'n_neighbors': 6, 'weights': 'uniform'}",0.226388,0.150351,0.076146,0.150914,0.250056,0.170771,0.061878,13
3,0.000798,0.000399,0.007579,0.000488,6,distance,"{'n_neighbors': 6, 'weights': 'distance'}",0.320819,0.256992,0.164710,0.260437,0.362410,0.273074,0.066965,2
4,0.000362,0.000448,0.006197,0.000723,7,uniform,"{'n_neighbors': 7, 'weights': 'uniform'}",0.136164,0.192242,0.077164,0.164597,0.241459,0.162325,0.054953,18


In [ ]:
knr_gs.best_params_, knr_gs.best_score_

({'n_neighbors': 10, 'weights': 'distance'}, 0.27441340092389555)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfr = RandomForestRegressor()
rfr_gs = GridSearchCV(
    rfr,
    {   
        "n_estimators": (8 , 16, 32, 64, 128, 256),
    }
)

In [ ]:
rfr_gs.fit(X_tr, y_tr)
rfr_df = pd.DataFrame(rfr_gs.cv_results_)
rfr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.030977,0.001561,0.001136,0.000445,8,{'n_estimators': 8},0.863831,0.923225,0.915406,0.869892,0.919614,0.898394,0.025935,5
1,0.053447,0.001018,0.002006,0.000013,16,{'n_estimators': 16},0.859975,0.910930,0.901864,0.866947,0.912811,0.890505,0.022498,6
2,0.105126,0.010302,0.003179,0.000749,32,{'n_estimators': 32},0.863968,0.920875,0.925860,0.879619,0.925305,0.903125,0.026114,2
3,0.204340,0.014336,0.004864,0.000777,64,{'n_estimators': 64},0.856954,0.923371,0.928749,0.873333,0.929887,0.902459,0.030983,4
4,0.430837,0.021800,0.009987,0.001670,128,{'n_estimators': 128},0.862111,0.919943,0.929261,0.876534,0.927674,0.903105,0.028135,3


In [ ]:
rfr_gs.best_params_, rfr_gs.best_score_

({'n_estimators': 256}, 0.9048408377007304)

In [ ]:
from sklearn.svm import SVR

In [ ]:
svr = SVR()
svr_gs = GridSearchCV(
    svr,
    {
        "C": (0.1, 1.0, 10),
        "epsilon": (0.1, 1.0, 10),
    }
)
svr_gs.fit(X_tr, y_tr)

svr_df = pd.DataFrame(svr_gs.cv_results_)
svr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_epsilon,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009369,0.001486,0.005388,0.001955,0.1,0.1,"{'C': 0.1, 'epsilon': 0.1}",0.000116,-0.057369,-0.201811,-0.029707,-0.115842,-0.080923,0.071524,9
1,0.007984,0.000621,0.003935,0.000644,0.1,1.0,"{'C': 0.1, 'epsilon': 1.0}",0.000116,-0.057370,-0.201811,-0.029701,-0.115827,-0.080919,0.071523,8
2,0.008379,0.000488,0.004378,0.000497,0.1,10,"{'C': 0.1, 'epsilon': 10}",0.000116,-0.056837,-0.201811,-0.029279,-0.116559,-0.080874,0.071692,7
3,0.009176,0.001162,0.004388,0.000488,1.0,0.1,"{'C': 1.0, 'epsilon': 0.1}",0.001278,-0.055183,-0.200572,-0.027467,-0.115196,-0.079428,0.071772,6
4,0.008629,0.001687,0.005003,0.001116,1.0,1.0,"{'C': 1.0, 'epsilon': 1.0}",0.001278,-0.055180,-0.200572,-0.027467,-0.115196,-0.079427,0.071773,5


In [ ]:
svr_gs.best_params_, svr_gs.best_score_

({'C': 10, 'epsilon': 0.1}, -0.0679771280241436)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [ ]:
pl = Pipeline(
    [
        ("mise_echelle", MinMaxScaler()),
        ("support_vecteurs", SVR()),
    ]
)

In [ ]:
pl_gs = GridSearchCV(
    pl,
    {
        "support_vecteurs__C": (0.1, 1.0, 10),
        "support_vecteurs__epsilon": (0.1, 1.0, 10),
    }
)

In [ ]:
pl_gs.fit(X_tr, y_tr)

pl_df = pd.DataFrame(pl_gs.cv_results_)
pl_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_support_vecteurs__C,param_support_vecteurs__epsilon,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008177,0.000977,0.004787,0.000747,0.1,0.1,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",0.000307,-0.057256,-0.201915,-0.029561,-0.115632,-0.080811,0.071610,9
1,0.008577,0.000790,0.005180,0.001463,0.1,1.0,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",0.000307,-0.057221,-0.201915,-0.029556,-0.115663,-0.080809,0.071616,8
2,0.008384,0.000484,0.003990,0.000891,0.1,10,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",0.000307,-0.056686,-0.201915,-0.029134,-0.116420,-0.080770,0.071787,7
3,0.008373,0.000489,0.004778,0.000980,1.0,0.1,"{'support_vecteurs__C': 1.0, 'support_vecteurs...",0.003189,-0.054037,-0.199610,-0.026008,-0.113175,-0.077928,0.071966,4
4,0.008773,0.000397,0.004791,0.000753,1.0,1.0,"{'support_vecteurs__C': 1.0, 'support_vecteurs...",0.003189,-0.054045,-0.199610,-0.026008,-0.113175,-0.077930,0.071965,5


In [ ]:
pl_gs.best_params_, pl_gs.best_score_

({'support_vecteurs__C': 10, 'support_vecteurs__epsilon': 10},
 -0.04766514608492325)

In [ ]:
from sklearn.neural_network import MLPRegressor
import numpy as np

In [ ]:
pln = Pipeline(
    [
        ("mise_echelle", MinMaxScaler()),
        ("neurones", MLPRegressor()),
    ]
)

In [ ]:
pln_gs = GridSearchCV(
    pln,
    {
        "neurones__alpha": 10.0 ** -np.arange(1, 7),
        'neurones__hidden_layer_sizes': ((25,), (50, ), (100,), (20, 20)),
    }
)

In [ ]:
pln_gs.fit(X_tr, y_tr)

pln_df = pd.DataFrame(pln_gs.cv_results_)
pln_df.head()

c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_neurones__alpha,param_neurones__hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.276860,0.008912,0.000604,0.000493,0.1,"(25,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-7.265260,-7.803490,-7.418769,-8.308542,-7.885371,-7.736286,0.367941,23
1,0.557897,0.022063,0.000923,0.000146,0.1,"(50,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-7.169583,-7.742505,-7.361349,-8.244487,-7.828612,-7.669307,0.375801,14
2,0.553783,0.028081,0.001001,0.000010,0.1,"(100,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-7.104080,-7.622723,-7.283237,-8.092599,-7.723177,-7.565163,0.345991,10
3,0.179531,0.012099,0.000395,0.000483,0.1,"(20, 20)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-6.613025,-7.484969,-6.646206,-7.246459,-7.058851,-7.009902,0.338771,3
4,0.270261,0.001663,0.000817,0.000409,0.01,"(25,)","{'neurones__alpha': 0.01, 'neurones__hidden_la...",-7.257464,-7.810626,-7.397353,-8.338590,-7.873456,-7.735498,0.382428,22


In [ ]:
pln_gs.best_params_

pln_gs.best_score_

-6.913083383837953

In [ ]:
pln_gs.best_params_

{'neurones__alpha': 0.0001, 'neurones__hidden_layer_sizes': (20, 20)}

In [ ]:
essai = en_gs.best_estimator_

In [ ]:
X

array([[2.0629e+04, 2.0210e+03, 1.1000e+02, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [5.1040e+03, 2.0220e+03, 1.8100e+02, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [8.9000e+03, 2.0210e+03, 1.3100e+02, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       ...,
       [3.0000e+01, 2.0220e+03, 1.1000e+02, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [3.0000e+01, 2.0220e+03, 1.1000e+02, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [3.0000e+01, 2.0220e+03, 1.1000e+02, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00]])

In [ ]:
prix_prédits = essai.predict(X)

In [ ]:
prix_reels = y

In [ ]:
prix_reels,prix_prédits

(array([22990, 42790, 23990, 17290, 17490, 26980, 37390, 17990, 18490,
        13790, 12280, 29790, 33790, 11480, 27290, 14990, 10990, 13990,
        31990, 23990, 10990, 23990, 19990, 11990, 24990, 23490, 34990,
        22990,  9790, 13990, 15990, 13790, 26790, 19390, 26790, 13990,
        24990, 26990, 27990, 22890, 14990, 21990, 26990, 15790, 15790,
        15790, 14990, 27790, 10990, 26990, 15490, 27990,  7490, 21990,
         9990, 18990, 11990, 18890, 19990, 19890, 13990, 22990, 13990,
        27990, 25990, 15980, 27490, 45990, 27490, 22990, 24990, 16990,
        15990, 20990, 26990, 26990, 43990, 14490, 24990, 19990, 18990,
        16490, 17990, 20990, 23990, 19490, 31990, 20490, 18990, 19790,
        18990,  9790, 33990, 17490, 16990, 36990, 13990, 27490, 13990,
        32990, 34990, 26890, 11990, 39000, 27900, 11500, 26900, 30900,
        31900, 29500, 49000, 18900, 18800, 17900, 22400, 25900, 16500,
        30500, 28500, 27900, 22500, 36500, 44990, 23900, 18900, 40900,
      

In [ ]:
len(prix_prédits)

600

In [ ]:
len(prix_reels)

600

In [ ]:
prix_prédits

array([22032.16884637, 41493.97568329, 24969.01072275, 17015.3782826 ,
       17126.27646582, 27114.10183904, 33576.76423118, 19583.24971472,
       18458.24464883, 16119.24664827, 11553.52919896, 28559.24319469,
       33350.51359537, 12961.59346727, 27344.15342119, 15604.64210085,
       10332.40695653, 15868.1933297 , 35350.51151928, 24947.14173634,
        9507.27973956, 24999.61094805, 20437.72395714, 16183.5785166 ,
       27850.37764051, 24385.08499141, 37405.14574699, 23939.98606217,
       11153.80307871, 11257.19463506, 16320.25719375, 12501.68882293,
       26683.83113381, 18697.69449216, 27204.38577592, 13768.77143029,
       26995.87023121, 31328.11651601, 30319.07857488, 22677.05061951,
       15929.1929971 , 21785.09065053, 27350.29152896, 17481.89883403,
       16997.2907344 , 17008.89317683, 16604.82248709, 28832.39067338,
       11051.50005201, 28996.1101355 , 15901.17853315, 26686.1155067 ,
        6650.5662699 , 23074.67351689, 11616.04192409, 20669.03596089,
      

In [ ]:
prix_reels

array([22990, 42790, 23990, 17290, 17490, 26980, 37390, 17990, 18490,
       13790, 12280, 29790, 33790, 11480, 27290, 14990, 10990, 13990,
       31990, 23990, 10990, 23990, 19990, 11990, 24990, 23490, 34990,
       22990,  9790, 13990, 15990, 13790, 26790, 19390, 26790, 13990,
       24990, 26990, 27990, 22890, 14990, 21990, 26990, 15790, 15790,
       15790, 14990, 27790, 10990, 26990, 15490, 27990,  7490, 21990,
        9990, 18990, 11990, 18890, 19990, 19890, 13990, 22990, 13990,
       27990, 25990, 15980, 27490, 45990, 27490, 22990, 24990, 16990,
       15990, 20990, 26990, 26990, 43990, 14490, 24990, 19990, 18990,
       16490, 17990, 20990, 23990, 19490, 31990, 20490, 18990, 19790,
       18990,  9790, 33990, 17490, 16990, 36990, 13990, 27490, 13990,
       32990, 34990, 26890, 11990, 39000, 27900, 11500, 26900, 30900,
       31900, 29500, 49000, 18900, 18800, 17900, 22400, 25900, 16500,
       30500, 28500, 27900, 22500, 36500, 44990, 23900, 18900, 40900,
       26900, 21000,

Objectif : maximiser le surplus de l'acheteur, i.e, si l'acheteur à un budget R, et n contraintes, quel véhicule maximiserait son surplus


In [ ]:
data_finale = conversion_df.data_frame_dummies(df)

In [ ]:
data_finale['prix_predits'] = prix_prédits
data_finale['surplus_conso'] = data_finale['prix'] - data_finale['prix_predits']

In [ ]:
data_finale

,prix,kilometrage,date_mise_circulation,puissance,nb_places,puissance_fiscale,critair,ptac,nb_portes,BERLINGO,...,SPOTICAR PREMIUM,Ex-Auto-école,Ex-Import,Ex-Loueur,Ex-Particulier,Ex-Société,Véhicule de direction,Citroen,prix_predits,surplus_conso
0,22990,20629.0,2021,110.0,5.0,6.0,2.0,1835.0,5.0,0,...,1,0,1,0,0,0,0,1,22032.168846,957.831154
1,42790,5104.0,2022,181.0,5.0,10.0,1.0,2300.0,5.0,0,...,1,0,0,1,0,0,0,1,41493.975683,1296.024317
2,23990,8900.0,2021,131.0,5.0,7.0,1.0,1735.0,5.0,0,...,1,0,1,0,0,0,0,1,24969.010723,-979.010723
3,17290,1305.0,2021,83.0,5.0,4.0,1.0,1540.0,5.0,0,...,1,0,1,0,0,0,0,1,17015.378283,274.621717
4,17490,37377.0,2019,110.0,5.0,5.0,1.0,1600.0,5.0,0,...,1,0,1,0,0,0,0,1,17126.276466,363.723534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,17579,10598.0,2018,110.0,5.0,5.0,1.0,1795.0,5.0,0,...,1,0,0,0,1,0,0,1,18157.420375,-578.420375
596,22689,30.0,2022,110.0,5.0,6.0,1.0,1610.0,5.0,0,...,1,0,1,0,0,0,0,1,22440.428349,248.571651
597,22689,30.0,2022,110.0,5.0,6.0,1.0,1610.0,5.0,0,...,1,0,1,0,0,0,0,1,22440.428349,248.571651
598,22689,30.0,2022,110.0,5.0,6.0,1.0,1610.0,5.0,0,...,1,0,1,0,0,0,0,1,22810.937683,-121.937683
